In [5]:
!pip install pytube


In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

JiWER is a simple and fast python package to evaluate an automatic speech recognition system.

In [13]:
import os 
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY =os.getenv("OPENAI_API_KEY")

In [14]:
from pytube import YouTube

In [15]:
link='https://www.youtube.com/watch?v=mBjPyte2ZZo'

In [18]:
try: 
    # object creation using YouTube
    yt = YouTube(link) 
except: 
    print("Connection Error")

In [19]:
yt.streams.filter(file_extension='mp4')

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001f" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="133" mime_type="video/mp4" res="240p" fps="30fps" vcodec="avc1.4d4015" progressive="False" type="video">, <Stream: itag="160" mime_type="video/mp4" res="144p" fps="30fps" vcodec="avc1.4d400c" progressive="False" type="video">, <Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">, <Stream: 

In [20]:
stream = yt.streams.get_by_itag(139)

In [21]:
stream.download('',"lecuninterview.mp4")

'/home/cloudsuperadmin/scrape-chain/lecuninterview.mp4'

In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("/home/cloudsuperadmin/scrape-chain/lecuninterview.mp4")
print(result['text'])

we got the result in a form of a raw text

In [23]:
with open ('text.txt', 'w') as file:  
    file.write(result['text']) 

In [24]:
from langchain import OpenAI, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain


In [30]:
from langchain import OpenAI
llm = OpenAI(temperature=0)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
    )

In [26]:
with open('text.txt') as f:
    text = f.read()
texts = text_splitter.split_text(text)

In [27]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:4]]

In [37]:
docs

[Document(page_content="Hi, I'm Craig Smith and this is I on A On. This week I talked to Jan LeCoon, one of the seminal figures in deep learning development and a long time proponent of self-supervised learning. Jan spoke about what's missing in large language models and about his new joint embedding predictive architecture which may be a step toward filling that gap. He also talked about his theory of consciousness and the potential for AI systems to someday exhibit the features of consciousness. It's a fascinating conversation that I hope you'll enjoy. Okay, so Jan, it's great to see you again. Good to see you again. I wanted to talk to you about where you've gone with just so supervised learning since last week spoke. In particular, I'm interested in how it relates to large language models because the large language models really came on stream since we spoke. In fact, in your talk about JEPA, which is joint embedding predictive architecture. Thank you. You mentioned that large lang

In [29]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [31]:
chain = load_summarize_chain(llm, 
                             chain_type="map_reduce")


output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

 Craig Smith interviews Jan LeCoon, a deep learning developer and proponent of self-supervised
learning, about his new joint embedding predictive architecture and his theory of consciousness.
Jan's research focuses on self-supervised learning and its use for pre-training transformer
architectures, which is used to predict missing words in a piece of text. Additionally, large
language models are used to predict the next word in a sentence, but it is difficult to represent
uncertain predictions when applying this to video.


In [32]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [33]:
chain = load_summarize_chain(llm, chain_type="stuff")

In [40]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS: AND TRANSLATE IT TO FRENCH"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

In [41]:
chain = load_summarize_chain(llm, 
                             chain_type="stuff", 
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary, 
                             width=2000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)



• Jan LeCoon is a seminal figure in deep learning development and a long-time proponent of self-supervised learning. 
• He discussed his new joint embedding predictive architecture which may be a step towards filling the gap in large language models. 
• He also discussed his theory of consciousness and the potential for AI systems to someday exhibit the features of consciousness. 
• Self-supervised learning has revolutionized natural language processing by using transformer architectures for pre-training. 
• Large language models lack a world model and are generative models, making it difficult to represent uncertain predictions. 

• Jan LeCoon est une figure emblématique du développement d'apprentissage profond et un partisan de longue date de l'apprentissage supervisé. 
• Il a discuté de sa nouvelle architecture d'apprentissage prédictif d'intégration conjointe qui pourrait être une étape vers le remplissage de l'écart dans les grands modèles de langage. 
• Il a également discuté d

In [42]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

  Craig Smith interviews Jan LeCoon, a deep learning developer and proponent of self-supervised
learning, about his new joint embedding predictive architecture and his theory of consciousness. Jan
discusses how large language models are missing something and how his architecture may be a step
towards filling that gap, as well as the potential for AI systems to exhibit features of
consciousness. Jan has gone on to revolutionize natural language processing with self-supervised
learning and transformer architectures, and explains how his joint embedding predictive architecture
fits into his research and how it relates to what's lacking in large language models. He explains
that self-supervised learning involves taking a piece of text, removing some of the words, and
replacing them with black markers, then training a neural net to predict the words that are missing.
This technique has been used in practical applications such as contact moderation systems on
Facebook, Google, YouTube, etc. 